In [243]:
import os
import pandas as pd
import plotly.graph_objects as go
import networkx as nx
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
import numpy as np
import plotly.express as px

base_dir = '/media/bigdata/projects/katz_pub_tree'
data_path = os.path.join(base_dir, 'data') 


In [162]:
paper_frame = pd.read_json(os.path.join(data_path,'katz_frame.json'))

In [163]:
# Determine y-position of points
# If year is even, offset rank by 0.5 so that marker texts don't overlap
grouped_frame = list(paper_frame.groupby('Date'))
for num,x in grouped_frame:
    if num%2==0:
        offset = 0
        x['year_even'] = '1'
    else:
        offset = 0.5
        x['year_even'] = '0'
    x['Rank'] = np.arange(x.shape[0]) + offset
    
paper_frame = pd.concat([x for num, x in grouped_frame])

In [164]:
paper_frame['First Author'] = [this_row[0].split(' ')[-1] for this_row in paper_frame['Authors']]

In [165]:
paper_frame['Summary'] = paper_frame['First Author'] + '\n' + paper_frame['Date'].astype('str')

In [166]:
# Limit authors for visualization
paper_frame['Authors_pretty'] = None
max_len = 4
for num,x in enumerate(paper_frame['Authors']):
    if len(x) > max_len:
        paper_frame['Authors_pretty'].iloc[num] = x[:4] + ['et al.']
    else:
         paper_frame['Authors_pretty'].iloc[num] = x

/home/abuzarmahmood/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [198]:
paper_frame['Fin_Article_Type'] = None
for num,these_types in enumerate(paper_frame['Article_Type']):
    temp_types = [x for x in these_types if 'Research' not in x]
    if len(temp_types) >1:
        temp_types = [x for x in temp_types if 'Journal Article' not in x]
    paper_frame['Fin_Article_Type'].iloc[num] = ",".join(temp_types)

/home/abuzarmahmood/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [236]:
max_title_len = 8 # words
title_words = [x.split(' ') for x in paper_frame['Title']]
title_split = [np.array_split(x, int(np.ceil(len(x)/max_title_len))) for x in title_words]
title_strs = [[" ".join(x) for x in y] for y in title_split]
title_pretty = ["<br>".join(x) for x in title_strs]
paper_frame['Title_Pretty'] = title_pretty

In [237]:
paper_frame

,Date,Title,PMID,Article_Type,Abstract,Authors,year_even,Rank,First Author,Summary,Authors_pretty,Fin_Article_Type,scatter_size,Title_Pretty
46,2002,Psychological functions of the cerebellum.,17715595,"[Journal Article, Review]","[For most of the 20th century, the brain scien...","[Donald B Katz, Joseph E Steinmetz]",1,0.0,Katz,Katz\n2002,"[Donald B Katz, Joseph E Steinmetz]",Review,40,Psychological functions of the cerebellum.
47,2002,Gustatory processing is dynamic and distributed.,12139994,"[Journal Article, Research Support, Non-U.S. G...",[The process of gustatory coding consists of n...,"[Donald B Katz, Miguel A L Nicolelis, Sidney A...",1,1.0,Katz,Katz\n2002,"[Donald B Katz, Miguel A L Nicolelis, Sidney A...",Review,40,Gustatory processing is dynamic and distributed.
45,2005,7 to 12 Hz activity in rat gustatory cortex re...,15574797,"[Comparative Study, Journal Article, Research ...",[The 7 to 12 Hz rhythm is a high-voltage oscil...,"[Alfredo Fontanini, Donald B Katz]",0,0.5,Fontanini,Fontanini\n2005,"[Alfredo Fontanini, Donald B Katz]",Comparative Study,40,7 to 12 Hz activity in rat gustatory<br>cortex...
41,2006,State-dependent modulation of time-varying gus...,16928791,"[Journal Article, Research Support, N.I.H., Ex...","[Sensory processing is modulated by attention,...","[Alfredo Fontanini, Donald B Katz]",1,0.0,Fontanini,Fontanini\n2006,"[Alfredo Fontanini, Donald B Katz]",Journal Article,40,State-dependent modulation of time-varying gus...
43,2006,Gustatory processing: a dynamic systems approach.,16842991,"[Journal Article, Research Support, N.I.H., Ex...",[Recent gustatory studies have provided a grow...,"[Lauren M Jones, Alfredo Fontanini, Donald B K...",1,1.0,Jones,Jones\n2006,"[Lauren M Jones, Alfredo Fontanini, Donald B K...",Review,40,Gustatory processing: a dynamic systems approach.
39,2007,Natural stimuli evoke dynamic sequences of sta...,18000059,"[Journal Article, Research Support, N.I.H., Ex...",[Although temporal coding is a frequent topic ...,"[Lauren M Jones, Alfredo Fontanini, Brian F Sa...",0,0.5,Jones,Jones\n2007,"[Lauren M Jones, Alfredo Fontanini, Brian F Sa...",Journal Article,40,Natural stimuli evoke dynamic sequences of<br>...
40,2007,The EJC factor eIF4AIII modulates synaptic str...,17632064,"[Journal Article, Research Support, N.I.H., Ex...",[Proper neuronal function and several forms of...,"[Corinna Giorgi, Gene W Yeo, Martha E Stone, D...",0,1.5,Giorgi,Giorgi\n2007,"[Corinna Giorgi, Gene W Yeo, Martha E Stone, D...",Journal Article,40,The EJC factor eIF4AIII modulates synaptic<br>...
35,2008,"Receptors, circuits, and behaviors: new direct...",19005043,"[Journal Article, Research Support, N.I.H., Ex...","[The chemical senses, smell and taste, are the...","[Donald B Katz, Hiroaki Matsunami, Dmitry Rinb...",1,0.0,Katz,Katz\n2008,"[Donald B Katz, Hiroaki Matsunami, Dmitry Rinb...",Review,40,"Receptors, circuits, and behaviors: new<br>dir..."
36,2008,"Behavioral states, network states, and sensory...",18614753,"[Journal Article, Research Support, N.I.H., Ex...",[We review data demonstrating that single-neur...,"[Alfredo Fontanini, Donald B Katz]",1,1.0,Fontanini,Fontanini\n2008,"[Alfredo Fontanini, Donald B Katz]",Review,40,"Behavioral states, network states, and sensory..."
37,2008,Investigating the motivational mechanism of al...,18410179,"[Journal Article, Research Support, N.I.H., Ex...",[The precise role played by serotonin (5-HT) i...,"[Melissa L Caras, Kimberly MacKenzie, Benjamin...",1,2.0,Caras,Caras\n2008,"[Melissa L Caras, Kimberly MacKenzie, Benjamin...",Journal Article,40,Investigating the motivational mechanism of al...


In [246]:
paper_frame['scatter_size'] = 40
fig = px.scatter(paper_frame, x="Date", y="Rank",
                 hover_name="Title", hover_data=['Date','Authors_pretty'],
                text = 'Summary', size = 'scatter_size',
                size_max = 50,
                width = 1500, height = 1000,
                color ='Fin_Article_Type',
                custom_data=['Title_Pretty','Authors_pretty'])

fig.update_traces(hovertemplate='<b>%{customdata[0]}</b>'
                         '<br>%{x}<br>%{customdata[1]}')
                #hovertemplate= ')#+
                #         '<br><b>Average Discharge</b>: %{lat}<br>'))
                #color_discrete_map = {'0':'silver','1':'gray'},
                #color = 'year_even',
                #symbol ='Fin_Article_Type')
                
fig.update_layout(uniformtext_minsize=6, 
                  uniformtext_mode='hide',
                 xaxis_title="Publication Date")
fig.show()

In [240]:
import plotly.io as pio
pio.write_html(fig, file='index.html', auto_open=True)

In [241]:
%ls

index.html       simple_network_graph.ipynb
query_pubmed.py  simple_network_graph.py


# =====================================

In [129]:
# #G = nx.random_geometric_graph(200, 0.125)
# G=nx.Graph()

# for num, this_row in paper_frame.iterrows():
#     G.add_node(num,pos=(this_row['Date'],this_row['Rank']))

In [72]:
# edge_x = []
# edge_y = []
# for edge in G.edges():
#     x0, y0 = G.nodes[edge[0]]['pos']
#     x1, y1 = G.nodes[edge[1]]['pos']
#     edge_x.append(x0)
#     edge_x.append(x1)
#     edge_x.append(None)
#     edge_y.append(y0)
#     edge_y.append(y1)
#     edge_y.append(None)

# edge_trace = go.Scatter(
#     x=edge_x, y=edge_y,
#     line=dict(width=0.5, color='#888'),
#     hoverinfo='none',
#     mode='lines')

# node_x = []
# node_y = []
# for node in G.nodes():
#     x, y = G.nodes[node]['pos']
#     node_x.append(x)
#     node_y.append(y)

# node_trace = go.Scatter(
#     x=node_x, y=node_y,
#     mode='markers + text',
#     hoverinfo='text',
#     marker=dict(
#         showscale=True,
#         # colorscale options
#         #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
#         #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
#         #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
#         colorscale='YlGnBu',
#         reversescale=True,
#         color=[],
#         size=10,
#         colorbar=dict(
#             thickness=15,
#             title='Node Connections',
#             xanchor='left',
#             titleside='right'
#         ),
#         line_width=2))

In [73]:
# # node_adjacencies = []
# # node_text = []
# # for node, adjacencies in enumerate(G.adjacency()):
# #     node_adjacencies.append(len(adjacencies[1]))
# #     node_text.append('# of connections: '+str(len(adjacencies[1])))

# # node_trace.marker.color = node_adjacencies

# # node_trace.text = node_text
# node_trace.text = paper_frame['Summary']

In [210]:
# fig = go.Figure(data=[edge_trace, node_trace],
#              layout=go.Layout(
#                 title='<br>Network graph made with Python',
#                 titlefont_size=16,
#                 showlegend=True,
#                 hovermode='closest',
#                 margin=dict(b=20,l=5,r=5,t=40),
#                 annotations=[ dict(
#                     showarrow=False,
#                     xref="paper", yref="paper",
#                     x=0.005, y=-0.002 ) ],
#                 xaxis=dict(showgrid=True, zeroline=False, showticklabels=True, autorange=True),
#                 yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
#                 )
# fig.show()